In [1]:
import pandas as pd
import numpy as np

from IPython.display import display
import datetime
import plotly.express as px
from plotly.colors import sequential
from plotly.colors import diverging

from plotly.subplots import make_subplots

In [2]:
matches_table = pd.read_csv('data/matches.csv')
display(matches_table)

,Date,Match Number,Stage,Player 1,Player 2,Score 1,Score 2
0,2023-11-18,1,Group,Evan Sooklal,Will Simpson,2,6
1,2023-11-18,2,Group,Paul Bartenfeld,Roman Ramirez,2,6
2,2023-11-18,3,Group,Roman Ramirez,Evan Sooklal,6,3
3,2023-11-18,4,Group,Will Simpson,Paul Bartenfeld,6,1
4,2023-11-18,5,Group,Paul Bartenfeld,Evan Sooklal,1,6
5,2023-11-18,6,Group,Will Simpson,Roman Ramirez,3,6
6,2023-11-18,7,Group,Will Simpson,Evan Sooklal,6,2
7,2023-11-18,8,Group,Roman Ramirez,Paul Bartenfeld,6,4
8,2023-11-18,9,Group,Evan Sooklal,Roman Ramirez,6,5
9,2023-11-18,10,Group,Paul Bartenfeld,Will Simpson,4,6


In [3]:
updated_time = f'<i>Updated {str(datetime.datetime.now().strftime("%A, %b %d, %Y %H:%M:%S"))} CT</i>'
link_fig = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/index.html'>ELO Graph</a>"
link_fig1 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig1.html'>H2H Stats</a>"
link_fig2 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig2.html'>H2H Advanced Stats</a>"
link_fig3 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig3.html'>H2H More Stats</a>"
link_fig4 = "<a href='https://htmlpreview.github.io/?https://github.com/notromanramirez/dgn_showdown/blob/main/figures/fig4.html'>Player Data</a>"

In [4]:
set_player1 = set(matches_table['Player 1'])
set_player2 = set(matches_table['Player 2'])
set_players = set_player1.union(set_player2)
num_players = len(set_players)
display(set_players)

{'Aaron Carter',
 'Evan Sooklal',
 'Paul Bartenfeld',
 'Roman Ramirez',
 'Will Simpson'}

In [5]:
# elo stuff
K = 31
base = 3

def expected_score(ratingA, ratingB, base=None):
    # game variables
    # base = 10
    normal_elo_difference = 100

    return (1 / (1 + np.power(base, (ratingB - ratingA) / normal_elo_difference)))

def rating_change(score, expected_score, K=None):
    # k-factor: determines how strongly a result affects the rating change
    # usually between 10 and 40, bit with a lot of games, we want to change it often
    # K = 32
    return K * (score - expected_score)

def sort_index_by_elo(table, elo):
    for i in range(2):
        table.sort_index(axis=i, key=(lambda x: [elo[y] for y in x.values]), inplace=True)

def add_elo_to_index(table):
    table.index= [f"{x}<br>{(elo[x]):,.0f}" for x in table.index]
    table.columns = [f"{x}<br>({(elo[x]):,.0f})" for x in table.columns]

def generate_elo(K, base):
    # ELO INITIALIZATION
    starting_elo = 1200.0
    elo = dict.fromkeys(set_players, starting_elo)
    elo_time = np.zeros([len(matches_table) + 1, num_players])
    elo_time_table = pd.DataFrame(elo_time)
    elo_time_table.columns = sorted(elo)
    elo_time_table.replace(0, np.NaN, inplace=True)

    prev_elo_time_table = elo_time_table.copy(deep=True)
    d_elo_time_table = elo_time_table.copy(deep=True)
    exp_elo_time_table = elo_time_table.copy(deep=True)
    wl_time_table = elo_time_table.copy(deep=True)

    record_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]),dtype=object)
    record_table.index = list(set_players)
    record_table.columns = list(set_players)

    luck_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]))
    luck_table.index = list(set_players)
    luck_table.columns = list(set_players)

    for c in record_table.columns:
        for r in record_table.index:
            record_table.at[r, c] = np.array([0, 0])

    plus_minus_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]),dtype=object)
    plus_minus_table.index = list(set_players)
    plus_minus_table.columns = list(set_players)
    
    for c in plus_minus_table.columns:
        for r in plus_minus_table.index:
            plus_minus_table.at[r, c] = np.array([0, 0])

    elo_time_table.iloc[0,:] = starting_elo
    prev_elo_time_table.iloc[0,:] = starting_elo

    for (i, row) in matches_table.iterrows():
        elo_p1 = elo[row['Player 1']]
        elo_p2 = elo[row['Player 2']]

        prev_elo_time_table.loc[i + 1, row['Player 1']] = elo_p1
        prev_elo_time_table.loc[i + 1, row['Player 2']] = elo_p2
        
        win_prob_p1 = expected_score(elo_p1, elo_p2, base=base)
        win_prob_p2 = expected_score(elo_p2, elo_p1, base=base)
        
        exp_elo_time_table.loc[i + 1, row['Player 1']] = win_prob_p1
        exp_elo_time_table.loc[i + 1, row['Player 2']] = win_prob_p2

        rating_change_p1 = rating_change(row['Score 1'] > row['Score 2'], win_prob_p1, K=K)
        rating_change_p2 = rating_change(row['Score 2'] > row['Score 1'], win_prob_p2, K=K)

        if rating_change_p1 > 0:
            luck_table.loc[row['Player 1'], row['Player 2']] += 1 - win_prob_p1
            luck_table.loc[row['Player 2'], row['Player 1']] += 0 - win_prob_p2
        elif rating_change_p2 > 0:
            luck_table.loc[row['Player 1'], row['Player 2']] += 0 - win_prob_p1
            luck_table.loc[row['Player 2'], row['Player 1']] += 1 - win_prob_p2

        plus_minus_table.loc[row['Player 1'], row['Player 2']][0] += row['Score 1']
        plus_minus_table.loc[row['Player 1'], row['Player 2']][1] += row['Score 2']
        plus_minus_table.loc[row['Player 2'], row['Player 1']][0] += row['Score 2']
        plus_minus_table.loc[row['Player 2'], row['Player 1']][1] += row['Score 1']

        if row['Score 1'] > row['Score 2']:
            record_table.loc[row['Player 1'], row['Player 2']][0] += 1
            record_table.loc[row['Player 2'], row['Player 1']][1] += 1
            wl_time_table.loc[i + 1, row['Player 1']] = row['Score 1'] - row['Score 2']
            wl_time_table.loc[i + 1, row['Player 2']] = row['Score 2'] - row['Score 1']
        elif row['Score 2'] > row['Score 1']:
            record_table.loc[row['Player 2'], row['Player 1']][0] += 1
            record_table.loc[row['Player 1'], row['Player 2']][1] += 1
            wl_time_table.loc[i + 1, row['Player 1']] = row['Score 1'] - row['Score 2']
            wl_time_table.loc[i + 1, row['Player 2']] = row['Score 2'] - row['Score 1']

        d_elo_time_table.loc[i + 1, row['Player 1']] = rating_change_p1
        d_elo_time_table.loc[i + 1, row['Player 2']] = rating_change_p2

        elo[row['Player 1']] += rating_change_p1
        elo[row['Player 2']] += rating_change_p2

        elo_time_table.loc[i + 1, row['Player 1']] = elo[row['Player 1']]
        elo_time_table.loc[i + 1, row['Player 2']] = elo[row['Player 2']]

    prev_elo_time_table = prev_elo_time_table.iloc[1:,:]
    elo_time_table = elo_time_table.iloc[1:,:]
    d_elo_time_table = d_elo_time_table.iloc[1:,:]
    exp_elo_time_table = exp_elo_time_table.iloc[1:,:]
    wl_time_table = wl_time_table.iloc[1:,:]

    list_players = sorted(set_players, key=lambda x: elo[x], reverse=False)

    for table in [record_table, luck_table, plus_minus_table]:
        sort_index_by_elo(table, elo)

    return (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, wl_time_table, record_table, luck_table, plus_minus_table)

(elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, wl_time_table, record_table, luck_table, plus_minus_table) = generate_elo(K=K,base=base)

luck_per_game_table = luck_table.copy(deep=True)

for c in luck_per_game_table.columns:
    for r in luck_per_game_table.index:
        if r !=c :
            luck_per_game_table.loc[r, c] /= record_table.loc[r, c].sum()
        else:
            luck_per_game_table.loc[r, c] = np.NaN


display(record_table)
display(luck_per_game_table)
display(plus_minus_table)
display(wl_time_table)

prev_elo_time_table.to_excel('exports/prev_elo_time_table.xlsx')
d_elo_time_table.to_excel('exports/d_elo_time_table.xlsx')
elo_time_table.to_excel('exports/elo_time_table.xlsx')
exp_elo_time_table.to_excel('exports/exp_elo_time_table.xlsx')



,Paul Bartenfeld,Roman Ramirez,Will Simpson,Evan Sooklal,Aaron Carter
Paul Bartenfeld,"[0, 0]","[2, 4]","[0, 6]","[1, 5]","[1, 3]"
Roman Ramirez,"[4, 2]","[0, 0]","[6, 3]","[1, 5]","[1, 3]"
Will Simpson,"[6, 0]","[3, 6]","[0, 0]","[6, 2]","[1, 3]"
Evan Sooklal,"[5, 1]","[5, 1]","[2, 6]","[0, 0]","[2, 5]"
Aaron Carter,"[3, 1]","[3, 1]","[3, 1]","[5, 2]","[0, 0]"


,Paul Bartenfeld,Roman Ramirez,Will Simpson,Evan Sooklal,Aaron Carter
Paul Bartenfeld,NaN,-0.054102,-0.301516,-0.179256,-0.061861
Roman Ramirez,0.054102,NaN,0.212328,-0.383739,-0.196001
Will Simpson,0.301516,-0.212328,NaN,0.188215,-0.309828
Evan Sooklal,0.179256,0.383739,-0.188215,NaN,-0.140090
Aaron Carter,0.061861,0.196001,0.309828,0.140090,NaN


,Paul Bartenfeld,Roman Ramirez,Will Simpson,Evan Sooklal,Aaron Carter
Paul Bartenfeld,"[0, 0]","[21, 31]","[16, 36]","[24, 35]","[15, 21]"
Roman Ramirez,"[31, 21]","[0, 0]","[47, 38]","[24, 33]","[18, 18]"
Will Simpson,"[36, 16]","[38, 47]","[0, 0]","[43, 26]","[14, 22]"
Evan Sooklal,"[35, 24]","[33, 24]","[26, 43]","[0, 0]","[26, 39]"
Aaron Carter,"[21, 15]","[18, 18]","[22, 14]","[39, 26]","[0, 0]"


,Aaron Carter,Evan Sooklal,Paul Bartenfeld,Roman Ramirez,Will Simpson
1,NaN,-4.0,NaN,NaN,4.0
2,NaN,NaN,-4.0,4.0,NaN
3,NaN,-3.0,NaN,3.0,NaN
4,NaN,NaN,-5.0,NaN,5.0
5,NaN,5.0,-5.0,NaN,NaN
6,NaN,NaN,NaN,3.0,-3.0
7,NaN,-4.0,NaN,NaN,4.0
8,NaN,NaN,-2.0,2.0,NaN
9,NaN,1.0,NaN,-1.0,NaN
10,NaN,NaN,-2.0,NaN,2.0


In [33]:
# player stats

player_table = pd.DataFrame(np.zeros([len(set_players), 0]))
player_table.index = set_players
player_table.sort_index(axis=0, key=(lambda x: [elo[y] for y in x.values]), ascending=False, inplace=True)

# ELO

player_table['Current ELO'] = [elo[x] for x in player_table.index]
player_table['Max ELO'] = elo_time_table.replace(to_replace=np.NaN, value=-np.inf).max(axis=0)
player_table['Min ELO'] = elo_time_table.replace(to_replace=np.NaN, value=np.inf).min(axis=0)

date2matches_len = dict.fromkeys(set(matches_table['Date']),0)
date2matches_first = dict.fromkeys(set(matches_table['Date']),0)
for (i, row) in matches_table.iterrows():
    if date2matches_len[row['Date']] == 0:
        date2matches_first[row['Date']] = i
    date2matches_len[row['Date']] += 1

# CHAMPIONSHIPS AND RUNNER UPS

championships = dict.fromkeys(set_players, 0)
runner_ups = dict.fromkeys(set_players, 0)
for date in set(matches_table['Date']):
    row = matches_table.iloc[date2matches_first[date] + date2matches_len[date] - 1, :]
    if row['Score 1'] > row['Score 2']:
        championships[row['Player 1']] += 1
        runner_ups[row['Player 2']] += 1
    elif row['Score 2'] > row['Score 1']:
        championships[row['Player 2']] += 1
        runner_ups[row['Player 1']] += 1

player_table['Titles'] = pd.DataFrame(championships.values(), index=championships.keys())
player_table['Runner-Ups'] = pd.DataFrame(runner_ups.values(), index=runner_ups.keys())

# RECORDS

player_table['Record'] = record_table.sum(axis=1)
player_table['GP'] = [sum(r) for r in player_table['Record']]
player_table['W'] = [r[0] for r in player_table['Record']]
player_table['L'] = [r[1] for r in player_table['Record']]
player_table['WL'] = player_table['W'] - player_table['L']
player_table['GB'] = (player_table['WL'] - player_table['WL'].max()) / 2
player_table['PCT'] = player_table['W'] / player_table['GP']
player_table.drop(columns='Record', inplace=True)

# POINTS

player_table['Raw +/-'] = plus_minus_table.sum(axis=1)
player_table['PD'] = [r[0] - r[1] for r in player_table['Raw +/-']]
player_table['PF'] = [r[0] for r in player_table['Raw +/-']]
player_table['PA'] = [r[1] for r in player_table['Raw +/-']]
player_table['PD/G'] = player_table['PD'] / player_table['GP']
player_table['PF/G'] = player_table['PF'] / player_table['GP']
player_table['PA/G'] = player_table['PA'] / player_table['GP']
player_table.drop(columns='Raw +/-', inplace=True)

# LUCK

player_table['Luck'] = luck_table.sum(axis=1)
player_table['Luck/G'] = player_table['Luck'] / player_table['GP']

# N-POINT GAMES
npg_table = pd.DataFrame(np.zeros([len(set_players), 0]))
npg_table.index = player_table.index
sort_index_by_elo(npg_table, elo)

for i in range(6):
    npg_table[f'{i+1}-PT G'] = [[0, 0] for x in range(len(set_players))]
for (i, row) in matches_table.iterrows():
    npg_table.loc[row['Player 1'], f"{abs(row['Score 1'] - row['Score 2'])}-PT G"][row['Score 1'] < row['Score 2']] += 1
    npg_table.loc[row['Player 2'], f"{abs(row['Score 1'] - row['Score 2'])}-PT G"][row['Score 1'] > row['Score 2']] += 1
display(npg_table)

# STREAKS

streaks = dict.fromkeys(set_players, list())

for player in streaks.keys():
    running = [0]
    for is_win in wl_time_table[player].dropna().reset_index(drop=True).apply(np.sign):
        if running[-1] == 0:
            running.append(is_win)
        elif (is_win > 0) and (running[-1] < 0):
            running.append(1)
        elif (is_win < 0) and (running[-1] > 0):
            running.append(-1)
        else:
            running.append(running[-1] + is_win)
    streaks[player] = running[1:]

streaks_table = pd.DataFrame(np.zeros([max([len(x) for x in streaks.values()]), len(set_players)]))
streaks_table.columns = set_players
streaks_table.sort_index(axis=1, key=(lambda x: [elo[y] for y in x.values]), ascending=False, inplace=True)

for player in streaks_table.columns:
    streaks_table[player] = pd.Series(streaks[player])

for x in ['1-PT G', '6-PT G']:
    player_table[x] = npg_table[x]
    player_table[x] = [f'{w}-{l}' for (w, l) in player_table[x]]


player_table['Streak'] = 0
for player in player_table.index:
    val = ""
    if streaks[player][-1] > 0:
        val += "W"
    elif streaks[player][-1] < 0:
        val += "L"
    val += str(int(np.abs(streaks[player][-1])))
    player_table.loc[player, 'Streak'] = val

player_table['L10'] = ""
for player in player_table.index:
    l10_w = 0
    l10_l = 0
    for i in range(10):
        if streaks[player][-1 - i] > 0:
            l10_w += 1
        elif streaks[player][-1 - i] < 0:
            l10_l += 1
    player_table.loc[player, 'L10'] = f"{l10_w}-{l10_l}"

# STYLIZING

player_table_styled = player_table.style.format(
    {'Current ELO': '{:.0f}',
    'Max ELO': '{:.0f}',
    'Min ELO': '{:.0f}',
    'WL': '{:+,.0f}',
    'GB': '{:,.1f}',
    'PCT': '{:,.3f}',
    'PD': '{:+.0f}',
    'PD/G': '{:+,.1f}',
    'PF/G': '{:,.1f}',
    'PA/G': '{:,.1f}',
    'Luck': '{:+,.1%}',
    'Luck/G': '{:+,.1%}'
    }
)


with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(player_table_styled)


player_table_styled.to_html('figures/fig4.html')

,1-PT G,2-PT G,3-PT G,4-PT G,5-PT G,6-PT G
Paul Bartenfeld,"[1, 2]","[1, 5]","[2, 4]","[0, 3]","[0, 4]","[0, 0]"
Roman Ramirez,"[0, 4]","[4, 4]","[3, 2]","[3, 3]","[1, 0]","[1, 0]"
Will Simpson,"[2, 1]","[5, 4]","[0, 4]","[6, 1]","[3, 0]","[0, 1]"
Evan Sooklal,"[5, 3]","[4, 0]","[2, 2]","[2, 7]","[1, 1]","[0, 0]"
Aaron Carter,"[3, 1]","[1, 2]","[6, 1]","[3, 0]","[0, 0]","[1, 1]"


,Current ELO,Max ELO,Min ELO,Titles,Runner-Ups,GP,W,L,WL,GB,PCT,PD,PF,PA,PD/G,PF/G,PA/G,Luck,Luck/G,1-PT G,6-PT G,Streak,L10
Aaron Carter,1301,1301,1182,1,0,19,14,5,+9,0.0,0.737,+27,100,73,+1.4,5.3,3.8,+325.1%,+17.1%,3-1,1-1,W2,9-1
Evan Sooklal,1228,1252,1172,0,2,27,14,13,+1,-4.0,0.519,-10,120,130,-0.4,4.4,4.8,+89.2%,+3.3%,5-3,0-0,L2,6-4
Will Simpson,1205,1269,1197,1,1,27,16,11,+5,-2.0,0.593,+20,131,111,+0.7,4.9,4.1,+16.5%,+0.6%,2-1,0-1,W1,5-5
Roman Ramirez,1174,1258,1147,1,0,25,12,13,-1,-5.0,0.480,+10,120,110,+0.4,4.8,4.4,-85.1%,-3.4%,0-4,1-0,L1,3-7
Paul Bartenfeld,1093,1184,1093,0,0,22,4,18,-14,-11.5,0.182,-47,76,123,-2.1,3.5,5.6,-345.7%,-15.7%,1-2,0-0,L8,2-8


In [7]:
import plotly.graph_objects as go
import plotly.colors as pc

fig = go.Figure()

# adding player
ranking = 1
trace_colors = pc.qualitative.Bold
for (i, (player, current_rating)) in enumerate(sorted(elo.items(), key=lambda x:x[1], reverse=True)):
    fig.add_trace(go.Scatter(
        x=elo_time_table.index,
        y=elo_time_table[player],
        name=f'#{ranking} ({current_rating:.0f}) {player}',
        mode='lines+markers',
        connectgaps=True,
        text=[
            f"<br><b>Pre-Game ELO: </b>{x[1]:.0f}<br><b>Win Probability: </b>{x[0]:.1%}<br><br><b>Change in ELO:</b> {x[2]:+.0f}<br>" for x in zip(
                exp_elo_time_table[player],
                prev_elo_time_table[player],
                d_elo_time_table[player]
            )
        ],
        line=dict(
            shape='hv',
            color=trace_colors[i % len(trace_colors)]
        )
    ))
    ranking += 1

# adding highlighting by tournament
tournaments = list(sorted(set(matches_table['Date'])))
vrect_colors = ['green', 'red', 'yellow', 'blue', 'orange']
for (i, tourney) in enumerate(tournaments):
    fig.add_vrect(
        annotation_text=tourney,
        annotation_position="top left",
        x0=matches_table['Date'][matches_table['Date'] == tourney].index[0] + 0.5,
        x1=matches_table['Date'][matches_table['Date'] == tourney].index[-1] + 1.5,
        fillcolor=vrect_colors[i % len(vrect_colors)],
        opacity=0.1,
        line_width=0
    )

fig.update_layout(
    title=f'<b>Pokémon Showdown ELO Rating System by Roman Ramirez</b><br>{updated_time}<br>{link_fig1}, {link_fig2}, {link_fig3}, {link_fig4}',
    xaxis_title='<b>Game Number</b>',
    yaxis_title='<b>ELO Rating</b>'
)

customdata = np.stack((
        list(matches_table['Player 1']),
        list(matches_table['Player 2']),
        list(matches_table['Score 1']),
        list(matches_table['Score 2']),
        matches_table['Date']
    ), axis=-1)
hovertemplate = (
    '<i>%{customdata[4]|%A, %B %d, %Y}, Game %{x}</i><br>' +
    '<b>%{fullData.name}</b><br><br>' + 
    '<b>%{customdata[0]} vs. %{customdata[1]}</b><br>' +
    '<b>Final Score:</b> %{customdata[2]}-%{customdata[3]}<br>' + 
    '%{text}' + 
    '<b>Post-Game ELO:</b> %{y:,.0f}<br>' +
    '<extra></extra>'
)

fig.update_traces(
    customdata=customdata,
    hovertemplate=hovertemplate,
    opacity=0.8,
    legendgrouptitle_text='<b>#<i>Rank</i> (<i>Current ELO</i>) <i>Player</i></b>'
)

fig.show()
fig.write_html("index.html")

In [8]:
# historical table, with record_table
win_pct_table = record_table.copy(deep=True)
text_record_table = record_table.copy(deep=True)

for c in win_pct_table.columns:
    for r in win_pct_table.index:
        if c == r:
            win_pct_table.loc[r, c] = np.NaN
            text_record_table.loc[r, c] = ""
        else:
            w, l = win_pct_table.loc[r, c]
            w_pct = (w) / (w + l) if (w + l) != 0 else 0
            win_pct_table.loc[r, c] = w_pct
            text_record_table.loc[r, c] = f"{w}-{l}<br>{w_pct:,.1%}"

win_prob_table = pd.DataFrame(np.zeros([len(set_players), len(set_players)]), dtype=object)
win_prob_table.columns = set_players
win_prob_table.index = set_players
sort_index_by_elo(win_prob_table, elo)

for c in win_prob_table.columns:
    for r in win_prob_table.index:
            if r != c:
                win_prob_table.loc[r, c] = expected_score(elo[r], elo[c], base=base)
            else:
                win_prob_table.loc[r, c] = np.NaN


In [9]:
fig1 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[f"<b>{x}</b>" for x in ["Historical Win Percentage", "Current Matchup Predictor"]],
    x_title=f"{link_fig}, {link_fig2}, {link_fig3}"
)

go_win_pct = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=win_pct_table,
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=text_record_table,
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

go_win_prob = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=win_prob_table,
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=win_prob_table.applymap(lambda x: "" if np.isnan(x) else f"{x:.1%}"),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig1.add_trace(go_win_pct, row=1, col=1)
fig1.add_trace(go_win_prob, row=1, col=2)

fig1.layout.yaxis1.title="<b>Self</b>"
fig1.layout.yaxis2.title="<b>Self</b>"
fig1.layout.xaxis1.title="<b>Other</b>"
fig1.layout.xaxis2.title="<b>Other</b>"

fig1.update_traces(hoverinfo='skip')

fig1.update_layout(
    title=f"<b>Head-to-Head Statistics</b>, {updated_time}"
)

fig1.show()
fig1.write_html('figures/fig1.html')

In [24]:
fig3 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[f"<b>{x}</b>" for x in ["Head-to-Head Plus Minus", "N-Game Historical Record"]],
    x_title=f"<br>{link_fig}, {link_fig1}, {link_fig2}"
)

text_npg_table = npg_table.copy(deep=True)

for c in npg_table.columns:
    for r in npg_table.index:
        if c == r:
            text_npg_table.loc[r, c] = ""
        else:
            w, l = npg_table.loc[r, c]
            if (w + l) == 0:
                text_npg_table.loc[r, c] = ""
            else:
                w_pct = (w) / (w + l) if (w + l) != 0 else 0
                text_npg_table.loc[r, c] = f"{w}-{l}<br>{w_pct:,.1%}"

def calc_win_pct(wl):
    if sum(wl) == 0:
        return np.NaN
    else:
        return (wl[0]) / (sum(wl))

go_ng = go.Heatmap(
    x=[x.replace(" ", "<br>")+"ames" for x in npg_table.columns],
    y=[x.replace(" ", "<br>") for x in npg_table.index],
    z=npg_table.applymap(calc_win_pct),
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=text_npg_table,
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

text_plus_minus_table = plus_minus_table.copy(deep=True)

for c in plus_minus_table.columns:
    for r in plus_minus_table.index:
        if c == r:
            text_plus_minus_table.loc[r, c] = ""
        else:
            w, l = plus_minus_table.loc[r, c]
            if (w + l) == 0:
                text_plus_minus_table.loc[r, c] = ""
            else:
                w_pct = (w) / (w + l) if (w + l) != 0 else 0
                text_plus_minus_table.loc[r, c] = f"{w}-{l}<br>{w_pct:,.1%}"

go_pm = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in plus_minus_table.columns],
    y=[x.replace(" ", "<br>") for x in plus_minus_table.index],
    z=plus_minus_table.applymap(calc_win_pct),
    zmax=1,
    zmid=0.5,
    zmin=0,
    text=text_plus_minus_table,
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig3.add_trace(go_pm, row=1, col=1)
fig3.add_trace(go_ng, row=1, col=2)

fig3.layout.yaxis1.title="<b>Self</b>"
fig3.layout.yaxis2.title="<b>Self</b>"
fig3.layout.xaxis1.title="<b>Other</b>"
fig3.layout.xaxis2.title="<b>Other</b>"

fig3.update_traces(
    hoverinfo='skip'
)

fig3.update_layout(
    title=f"<b>Head-to-Head Statistics</b>, {updated_time}"
)

fig3.show()
fig3.write_html('figures/fig3.html')

In [21]:
plus_minus_table.applymap(calc_win_pct)

,Paul Bartenfeld,Roman Ramirez,Will Simpson,Evan Sooklal,Aaron Carter
Paul Bartenfeld,0.000000,-0.192308,-0.384615,-0.186441,-0.166667
Roman Ramirez,0.192308,0.000000,0.105882,-0.157895,0.000000
Will Simpson,0.384615,-0.105882,0.000000,0.246377,-0.222222
Evan Sooklal,0.186441,0.157895,-0.246377,0.000000,-0.200000
Aaron Carter,0.166667,0.000000,0.222222,0.200000,0.000000


In [11]:
# difference in win percentage and matchup predictor
# positive number means overrated, negative number means underrated
# understanding: Player A is overrated/underrated against Player B

key_upset_table = -win_pct_table + win_prob_table

fig2 = make_subplots(
    rows=1, cols=2,
    subplot_titles=[
        "<b>Over/Underrated</b><br>How Much Better is the Win Probability Against Record", 
        "<b>Average Luck Per Game</b><br>Average Difference in Actual and Expected Performance"
    ],
    x_title=f"{link_fig}, {link_fig1}, {link_fig3}"
)

go_ou = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=key_upset_table,
    zmax=max(key_upset_table.max(axis=None), luck_per_game_table.max(axis=None)),
    zmid=0,
    zmin=min(key_upset_table.min(axis=None), luck_per_game_table.min(axis=None)),
    text=key_upset_table.applymap(lambda x: "" if np.isnan(x) else f"{x:+.1%}"),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

go_luck_pg = go.Heatmap(
    x=[x.replace(" ", "<br>") for x in win_prob_table.columns],
    y=[x.replace(" ", "<br>") for x in win_prob_table.index],
    z=luck_per_game_table,
    zmax=max(key_upset_table.max(axis=None), luck_per_game_table.max(axis=None)),
    zmid=0,
    zmin=min(key_upset_table.min(axis=None), luck_per_game_table.min(axis=None)),
    text=luck_per_game_table.applymap(lambda x: "" if np.isnan(x) else f"{x:+.1%}"),
    texttemplate="%{text}",
    colorscale=diverging.RdBu_r,
    hoverongaps=False
)

fig2.add_trace(go_ou, row=1, col=1)
fig2.add_trace(go_luck_pg, row=1, col=2)

fig2.layout.yaxis1.title="<b>Self</b>"
fig2.layout.yaxis2.title="<b>Self</b>"
fig2.layout.xaxis1.title="<b>Other</b>"
fig2.layout.xaxis2.title="<b>Other</b>"

fig2.update_traces(hoverinfo='skip')

fig2.update_layout(
    title=f"<b>Head-to-Head Advanced Statistics</b>, {updated_time}"
)

fig2.show()
fig2.write_html('figures/fig2.html')

In [28]:
go_player_table = go.Table(
    header=dict(
        values=[f'<b>{x}</b>' for x in player_table.columns],
        line_color='darkslategray',
        fill_color='grey'
    ),
    cells=dict(
        values=player_table_styled
    )
)

fig4 = go.Figure(
    go_player_table
)

fig.show()

ValueError: 
    Invalid value of type 'pandas.io.formats.style.Styler' received for the 'values' property of table.cells
        Received value: <pandas.io.formats.style.Styler object at 0x00000181CC2CA9A0>

    The 'values' property is an array that may be specified as a tuple,
    list, numpy array, or pandas Series

In [12]:
# # system optimization

# def optimization1():
#     index = [_ for _ in np.arange(1, 51, 1.0)]
#     columns = [_ for _ in np.arange(2, 11, 1.0)]

#     opt_table = pd.DataFrame(np.zeros([len(index), len(columns)]))
#     opt_table.index = index
#     opt_table.columns = columns

#     for base in opt_table.columns:
#         for K in opt_table.index:
#             (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, record_table, luck_table, plus_minus_table) = generate_elo(K=K,base=base)
#             num_upsets = 0
#             for i in elo_time_table.index:
#                 prev_elo_slice = prev_elo_time_table.loc[i,:].dropna().values
#                 elo_slice = elo_time_table.loc[i,:].dropna().values
#                 combo_elo = np.array([prev_elo_slice, elo_slice])
#                 is_upset = ((combo_elo[1, 0] - combo_elo[0, 0]) * (combo_elo[0, 1] - combo_elo[0, 0])) > 0
#                 if is_upset:
#                     num_upsets += 1
#             opt_table.loc[K,base] = num_upsets

#     fig = px.imshow(opt_table.T,labels=dict(y="Base", x="K-value", color="Number of Upsets"))#, text_auto=True, aspect="auto")
#     fig.update_layout(
#         title="<b>Total Number of Upsets against K-Value and Base</b>"
#     )
#     fig.show()
#     return opt_table
# opt1 = optimization1()
# fig.write_html("figures/opt1.html")

In [13]:
# # system optimization

# def optimization2():
#     index = [_ for _ in np.arange(1, 51, 1.0)]
#     columns = [_ for _ in np.arange(2, 11, 1.0)]

#     opt_table = pd.DataFrame(np.zeros([len(index), len(columns)]))
#     opt_table.index = index
#     opt_table.columns = columns

#     for base in opt_table.columns:
#         for K in opt_table.index:
#             (elo, elo_time_table, prev_elo_time_table, d_elo_time_table, exp_elo_time_table, record_table, luck_table, plus_minus_table) = generate_elo(K=K,base=base)
#             total_correctness = 0
#             for i in elo_time_table.index:
#                 d_elo_slice = d_elo_time_table.loc[i,:].dropna().values
#                 exp_elo_slice = exp_elo_time_table.loc[i,:].dropna().values
#                 correctness = (np.sign(d_elo_slice) * exp_elo_slice).sum()
#                 total_correctness += correctness
#             opt_table.loc[K,base] = total_correctness

#     fig = px.imshow(opt_table.T,labels=dict(y="Base", x="K-value", color="Total Correctness"))#, text_auto=False, aspect="auto")
#     fig.update_layout(
#         title="<b>Win Percentage Correctness against K-Value and Base</b>"
#     )
#     fig.show()
#     return opt_table
# opt2 = optimization2()
# fig.write_html("figures/opt2.html")

In [14]:
# def norm_minmax(dataframe):
#     return ((dataframe - dataframe.min(axis=None)) / (dataframe.max(axis=None) - dataframe.min(axis=None)))

# norm_opt1 = norm_minmax(opt1)
# norm_opt2 = norm_minmax(opt2)

# fig = px.imshow(norm_opt2.T - norm_opt1.T,labels=dict(y="Base", x="K-value", color="Normalized Optimization"), text_auto=False, aspect="auto")
# fig.update_layout(
#     title="<b>K Value and Base against Both Optimization Functions</b>"
# )
# fig.show()
# fig.write_html("figures/opt_total.html")